In [ ]:
!nvidia-smi

# Fast.ai 必要的安裝
FAST.AI參考設置資料
https://towardsdatascience.com/fastai-image-classification-32d626da20

In [ ]:
#accelerate pandas
# !pip install modin[ray]
# !pip install pandas

In [ ]:
from fastai import *
from fastai.vision import *
import pandas as pd

# Input Image
because there is no corresponding function in fastai, so we inheritance "ImageList" and alter it to read .csv.

In [ ]:
path = '../input/Kannada-MNIST//'
path
! ls {path} -l

In [ ]:
# df1 = pd.read_csv(path+'train.csv')
# df2 = pd.read_csv(path+'Dig-MNIST.csv')
# df = pd.concat([df1,df2],axis=0)
# df.to_csv('train.csv',index = False)

In [ ]:
# df.iloc[:,1:] = df.iloc[:,1:].replace(range(128,254),'255')
# df.iloc[:,1:] = df.iloc[:,1:].replace(range(1,127),'0')

In [ ]:
# df.to_csv('train.csv',index = False)

In [ ]:
class CustomImageItemList(ImageList):
    def open(self, fn):
        img = fn.reshape(28, 28)
        img = np.stack((img,)*3, axis=-1) # convert to 3 channels
        return Image(pil2tensor(img, dtype=np.float32))

    @classmethod
    def from_csv_custom(cls, path:PathOrStr, csv_name:str, imgIdx:int=1, header:str='infer', **kwargs) -> 'ItemList':
        df = pd.read_csv(Path(path)/csv_name, header=header)
        res = super().from_df(df, path=path, cols=0, **kwargs)
        # convert pixels to an ndarray
        res.items = df.iloc[:,imgIdx:].apply(lambda x: x.values / 783.0, axis=1).values
        return res

# Databunch(Fast.AI)
get_transforms() 則是fastai的內建方法，提供了適用於大多數計算機視覺任務的預設資料增強方案：

以0.5的概率隨機水平翻轉

以0.75的概率在-10與10度之間旋轉

以0.75的概率在1與1.1倍之間隨機放大

以0.75的概率隨機改變亮度和對比度

以0.75的概率進行隨機對稱扭曲


大多數卷積神經網路的輸入層形狀都是28、32、64、96、224、384、512之類的數字

In [ ]:
#best 0.986
# test = CustomImageItemList.from_csv_custom(path=path, csv_name='test.csv', imgIdx=1)
# data = (CustomImageItemList.from_csv_custom(path=path, csv_name='train.csv')
#                        .split_by_rand_pct(0.2)
#                        .label_from_df(cols='label')
#                        .add_test(test, label=0)
#                        .transform(get_transforms(do_flip = False, max_rotate = 0.), size=49)
#                        .databunch(bs=256, num_workers=16)
#                        .normalize(mnist_stats))
# data

In [ ]:
test = CustomImageItemList.from_csv_custom(path=path, csv_name='test.csv', imgIdx=1)
# DigMNIST = CustomImageItemList.from_csv_custom(path=path, csv_name='Dig-MNIST.csv')
data = (CustomImageItemList.from_csv_custom(path=path, csv_name='train.csv')
                       .split_by_rand_pct(0.2)
#                       .split_by_idx(list(range(60000,70240)))
                       .label_from_df(cols='label')
                       .add_test(test, label=0)
#                        .transform(get_transforms(do_flip = False))
                       .transform(get_transforms(do_flip = False, max_rotate = 0.,max_zoom = 1.), size=49)#, p_affine = 0.), size=49)
                       .databunch(bs=256, num_workers=16)
                       .normalize(mnist_stats))
data

In [ ]:
data.show_batch(rows=3, figsize=(12,9))

# Model
resnet 50

In [ ]:
arch = models.resnet50
# arch = models.resnet152
# arch = models.resnet18
arch

In [ ]:
!mkdir -p /tmp/.cache/torch/checkpoints
!cp ../input/resnet50/resnet50-19c8e357.pth /tmp/.cache/torch/checkpoints/resnet50-19c8e357.pth
# !cp ../input/resnet152/resnet152-b121ed2d.pth /tmp/.cache/torch/checkpoints/resnet152-b121ed2d.pth

In [ ]:
# !cd ../input/radam-pytorch/RAdam/
# !ls

In [ ]:
# !cp ../input/radam-pytorch/RAdam .
# import radam
# optar = partial(radam)

In [ ]:
#learn = cnn_learner(data, arch,pretrained = False, loss_func = nn.CrossEntropyLoss(), metrics=[error_rate,accuracy], model_dir='../kaggle/working').to_fp16()

# learn = cnn_learner(data, arch,pretrained = False, opt_func = optar, loss_func = nn.CrossEntropyLoss(), metrics=[error_rate,accuracy], model_dir='../kaggle/working').to_fp16()
#learn = cnn_learner(data, models.densenet161, metrics=[error_rate, accuracy], model_dir="/tmp/model/", pretrained=False)

In [ ]:
#FOR TESTING
learn = cnn_learner(data, arch,pretrained = False, loss_func = nn.MultiMarginLoss(), metrics=[error_rate,accuracy], model_dir='../kaggle/working')

# Learning
One-Cycle-Policy 大概有三个步骤：

我们逐渐将学习率从 lr_max / div_factor 提高到 lr_max，同时我们逐渐减少从 mom_max 到 mom_min 的动量(momentum)。

反向再做一次：我们逐渐将学习率从 lr_max 降低到 lr_max / div_factor，同时我们逐渐增加从 mom_min 到 mom_max 的动量。

我们进一步将学习率从 lr_max / div_factor 降低到 lr_max /（div_factor x 100），我们保持动力稳定在 mom_max。

作者：lai-bluejay
链接：https://hacpai.com/article/1552929502787
来源：黑客派
协议：CC BY-SA 4.0 https://creativecommons.org/licenses/by-sa/4.0/

In [ ]:
# lr = 1e-02

In [ ]:
# learn.lr_find()
# learn.recorder.plot(suggestion=True)

In [ ]:
# 0.9986
learn.fit_one_cycle(20)
learn.save('stage-1-50')

In [ ]:
# #TEST
# learn.fit_one_cycle(3,1e-02)

In [ ]:
# learn.unfreeze()
# learn.lr_find()
# learn.recorder.plot()

In [ ]:
# lr = slice(1e-04, 1e-03)

In [ ]:
# learn.fit_one_cycle(6,lr)

# Plot

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_top_losses(20, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interp.most_confused(min_val=2)

# Submission(with fast.ai predict())
with learn.data.test_ds in datablock fast.ai

In [ ]:
tmp_df = pd.read_csv(path+'sample_submission.csv')
tmp_df.head()

In [ ]:
for i in range(0,5000):
    img = learn.data.test_ds[i][0]
    tmp_df.loc[i]=[i,int(learn.predict(img)[1])]
tmp_df

In [ ]:
tmp_df.to_csv('submission.csv',index=False)